In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import joblib
import logging
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# sort and split

# Load the CSV file
file_path = '/Users/fabrizioferrari/Desktop/OG.csv'
df = pd.read_csv(file_path)

# Ensure 'Fecha Cierre' is in datetime format
df['Fecha Cierre'] = pd.to_datetime(df['Fecha Cierre'], format='%m/%d/%Y', errors='coerce')

# Ensure 'Fecha Colocacion' is in datetime format
df['Fecha Colocacion'] = pd.to_datetime(df['Fecha Colocacion'], format='%m/%d/%Y', errors='coerce')

df = df[df['Score'] != 0]

# Sort by 'Operacion' and then by 'Fecha Cierre' within each 'Operacion'
df_sorted = df.sort_values(by=['Operacion', 'Fecha Cierre'])

# Split the data into two DataFrames based on 'Fecha Colocacion' months
training_split = df_sorted[df_sorted['Fecha Colocacion'].dt.month <= 9]
test_split = df_sorted[df_sorted['Fecha Colocacion'].dt.month >= 10]

# Save the sorted DataFrame to new CSV files
training_set = '/Users/fabrizioferrari/Desktop/final boss/Training_Set.csv'
test_set = '/Users/fabrizioferrari/Desktop/final boss/Test_Set.csv'
training_split.to_csv(training_set, index=False)
test_split.to_csv(test_set, index=False)

print(f"Data has been sorted and saved to {training_set} and {test_set}")

/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_11626/1215289551.py:5: DtypeWarning: Columns (7,17,56,58,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Data has been sorted and saved to /Users/fabrizioferrari/Desktop/final boss/Training_Set.csv and /Users/fabrizioferrari/Desktop/final boss/Test_Set.csv


In [3]:
#test set completo con columnas
# Load the CSV file
file_path = '/Users/fabrizioferrari/Desktop/final boss/Test_Set.csv'
df = pd.read_csv(file_path)

# Ensure 'Fecha Cierre' is in datetime format
df['Fecha Cierre'] = pd.to_datetime(df['Fecha Cierre'], format='%Y-%m-%d', errors='coerce')

# Standardize 'Fecha Venta' column: replace typical non-null representations with NaN
df['Fecha Venta'].replace(['None', 'N/A', '', 'nan'], pd.NaT, inplace=True)

# Convert 'Fecha Venta' to datetime, setting errors='coerce' to handle any remaining non-datetime entries
df['Fecha Venta'] = pd.to_datetime(df['Fecha Venta'], errors='coerce')

# Initialize dictionaries to store 'Mora Final' and 'Atraso' columns for each 'Operacion'
mora_final_dict = {}
atraso_dict = {'Atraso30': {}, 'Atraso60': {}, 'Atraso90': {}, 'Atraso120': {}, 'Atraso150': {}, 'Atraso180': {}}

for operacion, group in df.groupby('Operacion'):
    # Determine if the loan was sold
    sold = group['Fecha Venta'].notna().any()
    estado_operacion_10 = (group['ESTADO_OPERACION'] == 10).any()
    estado_operacion_7 = (group['ESTADO_OPERACION'] == 7).any()
    closest_to_present = group.loc[group['Fecha Cierre'].idxmax()]
    moroso = closest_to_present['Atraso'] > 30

    if not sold and estado_operacion_10:
        mora_final = 0
    elif estado_operacion_7 and not moroso:
        mora_final = 0
    elif estado_operacion_7 and moroso:
        # Calculate the Mora Final
        min_cuotas_pend = group[group['CUOTAS_PEND'] > 0]['CUOTAS_PEND'].min()
        valor_cuota = group['Valor Cuota'].iloc[0]
        mora_final_antes_de_venta = min_cuotas_pend * valor_cuota
        mora_final = mora_final_antes_de_venta * 0.5
    else:
        # Calculate the Mora Final
        min_cuotas_pend = group[group['CUOTAS_PEND'] > 0]['CUOTAS_PEND'].min()
        valor_cuota = group['Valor Cuota'].iloc[0]
        mora_final_antes_de_venta = min_cuotas_pend * valor_cuota
        mora_final = mora_final_antes_de_venta * 0.82

    # Store the 'Mora Final' for each 'Operacion'
    mora_final_dict[operacion] = mora_final

    # Determine 'Atraso' levels for each 'Operacion'
    atraso_dict['Atraso30'][operacion] = int(group['Atraso'].max() > 30)
    atraso_dict['Atraso60'][operacion] = int(group['Atraso'].max() > 60)
    atraso_dict['Atraso90'][operacion] = int(group['Atraso'].max() > 90)
    atraso_dict['Atraso120'][operacion] = int(group['Atraso'].max() > 120)
    atraso_dict['Atraso150'][operacion] = int(group['Atraso'].max() > 150)
    atraso_dict['Atraso180'][operacion] = int(group['Atraso'].max() > 180)

# Add the new columns to the original DataFrame
df['Mora Final'] = df['Operacion'].map(mora_final_dict)
df['Pagare'] = df['Cant. Cuotas'] * df['Valor Cuota']
df['Mora Final %'] = df.apply(lambda row: 0 if row['Mora Final'] == 0 else row['Mora Final'] / row['Pagare'], axis=1)
df['Atraso30'] = df['Operacion'].map(atraso_dict['Atraso30'])
df['Atraso60'] = df['Operacion'].map(atraso_dict['Atraso60'])
df['Atraso90'] = df['Operacion'].map(atraso_dict['Atraso90'])
df['Atraso120'] = df['Operacion'].map(atraso_dict['Atraso120'])
df['Atraso150'] = df['Operacion'].map(atraso_dict['Atraso150'])
df['Atraso180'] = df['Operacion'].map(atraso_dict['Atraso180'])

# Save the modified DataFrame to a new CSV file
output_file_path = '/Users/fabrizioferrari/Desktop/final boss/Test_Set_Columnas.csv'
df.to_csv(output_file_path, index=False)

print(f"Data with 'Mora Final', 'Pagare', 'Mora Final %', and 'Atraso' columns saved to {output_file_path}")

/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_11626/4294853946.py:4: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Data with 'Mora Final', 'Pagare', 'Mora Final %', and 'Atraso' columns saved to /Users/fabrizioferrari/Desktop/final boss/Test_Set_Columnas.csv


In [4]:
#training set condensado con columnas

# Load the CSV file
file_path = '/Users/fabrizioferrari/Desktop/final boss/Training_Set.csv'
df = pd.read_csv(file_path)

# Ensure 'Fecha Cierre' is in datetime format
df['Fecha Cierre'] = pd.to_datetime(df['Fecha Cierre'], format='%Y-%m-%d', errors='coerce')

# Standardize 'Fecha Venta' column: replace typical non-null representations with NaN
df['Fecha Venta'].replace(['None', 'N/A', '', 'nan'], pd.NaT, inplace=True)

# Convert 'Fecha Venta' to datetime, setting errors='coerce' to handle any remaining non-datetime entries
df['Fecha Venta'] = pd.to_datetime(df['Fecha Venta'], errors='coerce')

# Initialize dictionaries to store 'Mora Final' and 'Atraso' columns for each 'Operacion'
mora_final_dict = {}
atraso_dict = {'Atraso30': {}, 'Atraso60': {}, 'Atraso90': {}, 'Atraso120': {}, 'Atraso150': {}, 'Atraso180': {}}

for operacion, group in df.groupby('Operacion'):
    # Determine if the loan was sold
    sold = group['Fecha Venta'].notna().any()
    estado_operacion_10 = (group['ESTADO_OPERACION'] == 10).any()
    estado_operacion_7 = (group['ESTADO_OPERACION'] == 7).any()
    closest_to_present = group.loc[group['Fecha Cierre'].idxmax()]
    moroso = closest_to_present['Atraso'] > 30

    if not sold and estado_operacion_10:
        mora_final = 0
    elif estado_operacion_7 and not moroso:
        mora_final = 0
    elif estado_operacion_7 and moroso:
        # Calculate the Mora Final
        min_cuotas_pend = group[group['CUOTAS_PEND'] > 0]['CUOTAS_PEND'].min()
        valor_cuota = group['Valor Cuota'].iloc[0]
        mora_final_antes_de_venta = min_cuotas_pend * valor_cuota
        mora_final = mora_final_antes_de_venta * 0.5
    else:
        # Calculate the Mora Final
        min_cuotas_pend = group[group['CUOTAS_PEND'] > 0]['CUOTAS_PEND'].min()
        valor_cuota = group['Valor Cuota'].iloc[0]
        mora_final_antes_de_venta = min_cuotas_pend * valor_cuota
        mora_final = mora_final_antes_de_venta * 0.82

    # Store the 'Mora Final' for each 'Operacion'
    mora_final_dict[operacion] = mora_final

    # Determine 'Atraso' levels for each 'Operacion'
    atraso_dict['Atraso30'][operacion] = int(group['Atraso'].max() > 30)
    atraso_dict['Atraso60'][operacion] = int(group['Atraso'].max() > 60)
    atraso_dict['Atraso90'][operacion] = int(group['Atraso'].max() > 90)
    atraso_dict['Atraso120'][operacion] = int(group['Atraso'].max() > 120)
    atraso_dict['Atraso150'][operacion] = int(group['Atraso'].max() > 150)
    atraso_dict['Atraso180'][operacion] = int(group['Atraso'].max() > 180)

# Add the new columns to the original DataFrame
df['Mora Final'] = df['Operacion'].map(mora_final_dict)
df['Pagare'] = df['Cant. Cuotas'] * df['Valor Cuota']
df['Mora Final %'] = df.apply(lambda row: 0 if row['Mora Final'] == 0 else row['Mora Final'] / row['Pagare'], axis=1)
df['Atraso30'] = df['Operacion'].map(atraso_dict['Atraso30'])
df['Atraso60'] = df['Operacion'].map(atraso_dict['Atraso60'])
df['Atraso90'] = df['Operacion'].map(atraso_dict['Atraso90'])
df['Atraso120'] = df['Operacion'].map(atraso_dict['Atraso120'])
df['Atraso150'] = df['Operacion'].map(atraso_dict['Atraso150'])
df['Atraso180'] = df['Operacion'].map(atraso_dict['Atraso180'])

# Condense the DataFrame by keeping only the first row of each 'Operacion'
df_condensed = df.drop_duplicates(subset='Operacion', keep='first')

# Save the modified DataFrame to a new CSV file
output_file_path = '/Users/fabrizioferrari/Desktop/final boss/Training_Set_Columnas_Condensed.csv'
df_condensed.to_csv(output_file_path, index=False)

print(f"Data with 'Mora Final', 'Pagare', 'Mora Final %', 'Atraso30', 'Atraso60', 'Atraso90', 'Atraso120', 'Atraso150', and 'Atraso180' saved to {output_file_path}")


/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_11626/3529821944.py:5: DtypeWarning: Columns (17,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Data with 'Mora Final', 'Pagare', 'Mora Final %', 'Atraso30', 'Atraso60', 'Atraso90', 'Atraso120', 'Atraso150', and 'Atraso180' saved to /Users/fabrizioferrari/Desktop/final boss/Training_Set_Columnas_Condensed.csv


In [5]:
#generar columnas y condensar:
#columnas test set condensado stripped

# Define the columns to retain
categorical_cols = [ "Banca", "CALIFICACION_ANTERIOR",  
                        "MARCA",  "SEXO", "Tipo", "Departamento",  "Medio",  "Canal"]
numerical_cols = ["Cant. Cuotas", "Capital actual", "Edad", "INGRESO_CLIENTE", "Valor Cuota"]

# Load the CSV file
file_path = '/Users/fabrizioferrari/Desktop/final boss/Test_Set.csv'
df = pd.read_csv(file_path)

# Combine numerical and categorical columns with 'Operacion' as the key column to retain
cols_to_keep = ['Operacion'] + numerical_cols + categorical_cols

# Strip the unnecessary columns
df_reduced = df[cols_to_keep]

# Condense the DataFrame by keeping only the first row of each 'Operacion'
df_condensed = df_reduced.drop_duplicates(subset='Operacion', keep='first')

# Save the modified DataFrame to a new CSV file
output_file_path = '/Users/fabrizioferrari/Desktop/final boss/Test_Set_Condensed_Stripped.csv'
df_condensed.to_csv(output_file_path, index=False)

print(f"Data with selected columns saved to {output_file_path}")


/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_11626/2269864719.py:11: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Data with selected columns saved to /Users/fabrizioferrari/Desktop/final boss/Test_Set_Condensed_Stripped.csv


In [6]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def load_data(file_path, delimiter=','):
    """Load the CSV file with the correct delimiter and strip spaces from column names."""
    try:
        data = pd.read_csv(file_path, delimiter=delimiter)
        data.columns = data.columns.str.strip()
        return data
    except Exception as e:
        logging.error(f"Error loading data from {file_path}: {e}")
        raise

def filter_data(data):
    """Apply filtering conditions on the data."""
    filtered_data = data[(data['Score'] != 0 )]
    filtered_data = filtered_data[filtered_data['SEXO'].isin(['M', 'F'])]
    filtered_data = filtered_data[(filtered_data['Edad'] <= 90) & (filtered_data['Edad'] >= 18)]
    filtered_data = filtered_data[(filtered_data['INGRESO_CLIENTE'] <= 500000001) & (filtered_data['INGRESO_CLIENTE'] >= 1000000)]
    filtered_data = filtered_data[(filtered_data['Cant. Cuotas'] <= 24) & (filtered_data['Cant. Cuotas'] >= 1)]
    filtered_data = filtered_data[(filtered_data['Capital actual'] <= 30000000) & (filtered_data['Capital actual'] >= 300000)]
    filtered_data = filtered_data[(filtered_data['Valor Cuota'] <= 10000000) & (filtered_data['Valor Cuota'] >= 50000)]
    filtered_data = filtered_data[filtered_data['Banca'].isin([240, 420, 130, 471, 421, 470])]
    filtered_data = filtered_data[filtered_data['Tipo'].isin([201, 205, 300, 305, 200])]
    return filtered_data

def drop_columns(data, columns_to_drop):
    """Drop unnecessary columns from the data."""
    columns_to_drop = [col for col in columns_to_drop if col in data.columns]
    return data.drop(columns=columns_to_drop)

def preprocess_data(data, categorical_cols, numerical_cols, target_col):
    """Encode categorical variables, handle missing values, and scale numerical features."""
    data_encoded = pd.get_dummies(data, columns=categorical_cols)
    
    X = data_encoded.drop(columns=[target_col])
    y = data_encoded[target_col]
    
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    
    return X_scaled, y, imputer, scaler

In [13]:


def load_data(file_path):
    """Load the dataset from the specified file path."""
    return pd.read_csv(file_path)

def filter_data(data):
    """Apply any necessary data filtering here."""
    # Implement your data filtering logic
    return data

def drop_columns(data, columns):
    """Drop the specified columns from the dataset."""
    return data.drop(columns=columns, errors='ignore')

def preprocess_data(data, categorical_cols, numerical_cols, target_col):
    """Encode categorical variables, handle missing values, and scale numerical features."""
    # Encode categorical variables
    data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

    # Check if target column is present
    if target_col not in data_encoded.columns:
        logging.error(f"Target column '{target_col}' not found in data columns: {data_encoded.columns}")
        raise KeyError(f"Target column '{target_col}' not found in data columns")

    X = data_encoded.drop(columns=[target_col])
    y = data_encoded[target_col]

    # Check for non-numeric data in X
    non_numeric_columns = X.select_dtypes(include=['object']).columns.tolist()
    if non_numeric_columns:
        logging.error(f"Non-numeric data found in columns: {non_numeric_columns}")
        raise ValueError(f"Non-numeric data found in columns: {non_numeric_columns}")

    # Check for unique values in each column
    for col in non_numeric_columns:
        unique_values = X[col].unique()
        logging.info(f"Column {col} has unique values: {unique_values}")

    # Handle missing values
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)

    # Scale numerical features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)

    return X_scaled, y, imputer, scaler


def train_model(X, y):
    """Train the CatBoostClassifier with the specified parameters."""
    model = CatBoostClassifier(depth=5, iterations=400, l2_leaf_reg=3, learning_rate=0.06, random_seed=42, logging_level='Verbose')
    model.fit(X, y)
    return model

def save_model(model, imputer, scaler, model_path, imputer_path, scaler_path):
    """Save the trained model, imputer, and scaler to disk."""
    model.save_model(model_path)
    joblib.dump(imputer, imputer_path)
    joblib.dump(scaler, scaler_path)
    logging.info("Model, imputer, and scaler saved successfully.")

def main():
    # File paths
    train_file_path = '/Users/fabrizioferrari/Desktop/final boss/Training_Set_Columnas_Condensed.csv'
    model_path = '/Users/fabrizioferrari/Desktop/final boss/catboost_model.cbm'
    imputer_path = '/Users/fabrizioferrari/Desktop/final boss/imputer.pkl'
    scaler_path = '/Users/fabrizioferrari/Desktop/final boss/scaler.pkl'
    
    # Columns to drop
    columns_to_drop = [
    "Año de Fecha Cierre", "Fecha Cierre", "Fecha Colocacion", "Analista", 
    "APORTA_IVA", "Aportaips", "Aproblinea", "Aprobscoring", "Atraso", 
    "CALIFICACION","CIUDADLAB",'CIRCUITO_OPE', "CLIENTEFORMAL", "COBROWALTON", "COD_EMPRESA1_LAB", 
    "COD_EMPRESA2_LAB", "Condicionado", "Controlscoring", "CUENTA", 
    "CUOTAS_PEND", "CUOTASPAGADAS", "EMPRESA_PUBLICA1_LAB", 
    "EMPRESA_PUBLICA2_LAB", "EMPRESA1_LAB", "EMPRESA2_LAB", 
    "ESTADO_OPERACION", "Excepcion", "EXCEPCIONANALISTA", 
    "EXCEPCIONESTADO", "EXCEPCIONINSTANCIA", "EXCEPCIONMOTIVO", 
    "EXCEPCIONTIPO", "Faja", "Fecha Venta", "FECHA_CANCELACION", 
    "Franquicia", "HABILITA_PROD1_BNF", "HABILITA_PROD2_BNF", 
    "INTERES_VTA", "INTERES2", "IVA_LEY", "MONTO_ANTERIOR", 
    "MONTODESEMBOLSADO", "OPE_NUEVA", "OPEPARALELA", "Operacion", 
    "OPERACIONIPS", "PATENTE_COMERCIAL", "Rechazocarga", "RUC_EMPRESA1_LAB", 
    "RUC_EMPRESA2_LAB", "Saldo Capital", "Score", "SCORE_BICSA", 
    "SCORE_DATALAB", "SECTOR_ECONOMICO", "SITUACION", "Sucursal", 
    "Sucursaltipo", "Supervisor","Tipo_Aprobacion", "ULTIMO_ATRASO", 
    "Vendedor","Pagare","Mora Final %","Mora Final","Atraso30","Atraso60","Atraso120","Atraso150","Atraso90","Capital anterior","Capital Venta"
]

    # Categorical and numerical columns
    categorical_cols = [ "Banca", "CALIFICACION_ANTERIOR",  
                        "MARCA",  "SEXO", "Tipo", "Departamento",  "Medio",  "Canal"]
    numerical_cols = ["Cant. Cuotas", "Capital actual", "Edad", "INGRESO_CLIENTE", "Valor Cuota"]
    
    target_col = 'Atraso180'

    # Load and preprocess data
    logging.info("Loading data...")
    train_data = load_data(train_file_path)
    logging.info(f"Columns in the dataset after loading: {train_data.columns.tolist()}")
    
    logging.info("Filtering data...")
    train_data = filter_data(train_data)
    
    logging.info("Dropping unnecessary columns...")
    train_data = drop_columns(train_data, columns_to_drop)
    logging.info(f"Columns in the dataset after dropping: {train_data.columns.tolist()}")
    
    logging.info("Preprocessing data...")
    X_train, y_train, imputer, scaler = preprocess_data(train_data, categorical_cols, numerical_cols, target_col)
    
    # Train the model
    logging.info("Training the model...")
    final_model = train_model(X_train, y_train)

    # Save the model, imputer, and scaler
    logging.info("Saving the model, imputer, and scaler...")
    save_model(final_model, imputer, scaler, model_path, imputer_path, scaler_path)

    logging.info("Model training completed and saved.")

if __name__ == "__main__":
    main()


2024-07-29 14:49:34,697 - INFO - Loading data...
/var/folders/6m/k7tj7j8j0z1_ry92lnh0blg00000gn/T/ipykernel_11626/56561933.py:3: DtypeWarning: Columns (17,60) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)
2024-07-29 14:49:35,737 - INFO - Columns in the dataset after loading: ['Año de Fecha Cierre', 'Fecha Cierre', 'Fecha Colocacion', 'Analista', 'APORTA_IVA', 'Aportaips', 'Aproblinea', 'Aprobscoring', 'Atraso', 'Banca', 'CALIFICACION', 'CALIFICACION_ANTERIOR', 'Canal', 'Cant. Cuotas', 'Capital actual', 'Capital anterior', 'Capital Venta', 'CIRCUITO_OPE', 'CIUDADLAB', 'CLIENTEFORMAL', 'COBROWALTON', 'COD_EMPRESA1_LAB', 'COD_EMPRESA2_LAB', 'Condicionado', 'Controlscoring', 'CUENTA', 'CUOTAS_PEND', 'CUOTASPAGADAS', 'Departamento', 'Edad', 'EMPRESA_PUBLICA1_LAB', 'EMPRESA_PUBLICA2_LAB', 'EMPRESA1_LAB', 'EMPRESA2_LAB', 'ESTADO_OPERACION', 'Excepcion', 'EXCEPCIONANALISTA', 'EXCEPCIONESTADO', 'EXCEPCIONINSTANCIA', 'EXCEPCIONMOTIVO', 

0:	learn: 0.6676399	total: 9.06ms	remaining: 3.61s
1:	learn: 0.6447410	total: 17.5ms	remaining: 3.49s
2:	learn: 0.6258807	total: 26.4ms	remaining: 3.5s
3:	learn: 0.6086107	total: 36.2ms	remaining: 3.58s
4:	learn: 0.5944453	total: 58.1ms	remaining: 4.59s
5:	learn: 0.5808474	total: 68ms	remaining: 4.47s
6:	learn: 0.5696779	total: 77.7ms	remaining: 4.36s
7:	learn: 0.5598600	total: 86.9ms	remaining: 4.26s
8:	learn: 0.5509337	total: 96.3ms	remaining: 4.18s
9:	learn: 0.5431479	total: 106ms	remaining: 4.12s
10:	learn: 0.5365304	total: 115ms	remaining: 4.06s
11:	learn: 0.5304087	total: 125ms	remaining: 4.03s
12:	learn: 0.5250058	total: 133ms	remaining: 3.96s
13:	learn: 0.5199489	total: 142ms	remaining: 3.92s
14:	learn: 0.5157679	total: 152ms	remaining: 3.89s
15:	learn: 0.5121518	total: 161ms	remaining: 3.86s
16:	learn: 0.5088301	total: 169ms	remaining: 3.81s
17:	learn: 0.5060545	total: 178ms	remaining: 3.78s
18:	learn: 0.5033655	total: 186ms	remaining: 3.74s
19:	learn: 0.5013809	total: 195ms	r

2024-07-29 14:49:41,750 - INFO - Saving the model, imputer, and scaler...
2024-07-29 14:49:41,755 - INFO - Model, imputer, and scaler saved successfully.
2024-07-29 14:49:41,755 - INFO - Model training completed and saved.


397:	learn: 0.4611770	total: 5.4s	remaining: 27.1ms
398:	learn: 0.4611493	total: 5.41s	remaining: 13.6ms
399:	learn: 0.4611193	total: 5.42s	remaining: 0us


In [14]:
# Load the trained model, imputer, and scaler
model_path = '/Users/fabrizioferrari/Desktop/final boss/catboost_model.cbm'
final_model = CatBoostClassifier()
final_model.load_model(model_path)

imputer = joblib.load('/Users/fabrizioferrari/Desktop/final boss/imputer.pkl')
scaler = joblib.load('/Users/fabrizioferrari/Desktop/final boss/scaler.pkl')

# Load the new CSV file for testing
test_file_path = '/Users/fabrizioferrari/Desktop/final boss/Test_Set_Condensed_Stripped.csv'
test_data = pd.read_csv(test_file_path, delimiter=',')
test_data.columns = test_data.columns.str.strip()  # Strip spaces from column names

# Extract the 'Operacion' column for final output and ensure alignment
test_operacion_column = test_data['Operacion']

# Apply filtering conditions on the test data (ensure consistency with training data)
test_data_filtered = test_data.copy()  # Create a copy for alignment
test_data_filtered = test_data_filtered[test_data_filtered['SEXO'].isin(['M', 'F'])]
test_data_filtered = test_data_filtered[(test_data_filtered['Edad'] <= 90) & (test_data_filtered['Edad'] >= 18)]
test_data_filtered = test_data_filtered[(test_data_filtered['INGRESO_CLIENTE'] <= 500000001) & (test_data_filtered['INGRESO_CLIENTE'] >= 1000000)]
test_data_filtered = test_data_filtered[(test_data_filtered['Cant. Cuotas'] <= 24) & (test_data_filtered['Cant. Cuotas'] >= 1)]
test_data_filtered = test_data_filtered[(test_data_filtered['Capital actual'] <= 30000000) & (test_data_filtered['Capital actual'] >= 300000)]
test_data_filtered = test_data_filtered[(test_data_filtered['Valor Cuota'] <= 10000000) & (test_data_filtered['Valor Cuota'] >= 50000)]
test_data_filtered = test_data_filtered[test_data_filtered['Banca'].isin([240, 420, 130, 471, 421, 470])]
test_data_filtered = test_data_filtered[test_data_filtered['Tipo'].isin([201, 205, 300, 305, 200])]

print(test_data_filtered.head())

# Ensure that 'Operacion' is aligned with the filtered data
test_operacion_column = test_operacion_column.loc[test_data_filtered.index]

# Categorical columns to encode
categorical_cols = [
    "Banca", "CALIFICACION_ANTERIOR", "MARCA", "SEXO", "Tipo", "Departamento", "Medio", "Canal"
]

# Encode categorical variables in the test data
test_data_encoded = pd.get_dummies(test_data_filtered, columns=categorical_cols)

# Align the test data with the training data (imputed features)
missing_cols = set(imputer.feature_names_in_) - set(test_data_encoded.columns)
for col in missing_cols:
    test_data_encoded[col] = 0
test_data_encoded = test_data_encoded[imputer.feature_names_in_]

# Handle missing values in the test data
X_test_imputed = imputer.transform(test_data_encoded)

# Feature Scaling for the test data
X_test_scaled = scaler.transform(X_test_imputed)

# Make predictions on the test data
test_predictions = final_model.predict(X_test_scaled)

# Calculate the confidence scores
test_confidences = 1000 - final_model.predict_proba(X_test_scaled)[:, 1] * 1000  # Scale to 0-1000

# Create a DataFrame with the test set actual and predicted values
results_df = pd.DataFrame({
    'Operacion': test_operacion_column,
    'Confidence': test_confidences
})

# Save the results DataFrame to a new CSV file
output_file_path = '/Users/fabrizioferrari/Desktop/final boss/new_predictions.csv'
results_df.to_csv(output_file_path, index=False)

print("Predictions completed and saved.")
print(results_df.head(10))


   Operacion  Cant. Cuotas  Capital actual  Edad  INGRESO_CLIENTE  \
0   14515791            12         2357000    30        2550307.0   
1   14518998            18         1178500    28        2000000.0   
2   14522924            10         3535500    54        2000000.0   
3   14525445            12         2946250    57        2289324.0   
4   14526233            10         1178500    48        3500000.0   

   Valor Cuota  Banca CALIFICACION_ANTERIOR  MARCA SEXO  Tipo  \
0       254000    420                   EXE      4    M   205   
1       101000    130                   NaN      1    F   201   
2       435000    130                   EXE      1    M   305   
3       317000    240                   EXE      1    M   305   
4       147000    420                   EXE      4    M   205   

           Departamento  Medio  \
0  CENTRAL                  74   
1  CENTRAL                  95   
2  ALTO PARANA              96   
3  AMAMBAY                  96   
4  CENTRAL              